In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
function norm_dist(a::Array{Float64,1}, b::Array{Float64,1}, sigma::Float64)
    return exp(-((a[1] - b[1])^2 + (a[2] - b[2])^2) / 2 / sigma^2) / (2*pi) / sigma^2
end

In [ ]:
#load borders
border = import_border("../data/border.json")

# create the lattice grid
dx = 20
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))

# define x and y coordinate for plotting
x = repeat(reshape(xrange,1,Nx), Ny, 1)
y = repeat(reshape(yrange,Ny,1), 1, Nx);

In [ ]:
#compute 
data = h5read("../data/pantagruel.h5", "/")
mg = vec(data["gen_inertia"])
dg = vec(data["gen_prim_ctrl"])
idgen = Int64.(vec(data["gen"][:, 1]))
isgrid = isinside .| isborder
dl = vec(data["load_freq_coef"])
idb = Int64.(data["branch"][:, 1:2])
bline = 1 ./ data["branch"][:, 4]
coord = alberts_projection(
    data["bus_coord"] ./ (180 / pi),
    13.37616 / 180 * pi,
    46.94653 / 180 * pi,
    10 / 180 * pi,
    50 / 180 * pi,
)
    
dem = vec(data["bus"][:, 3]) / 100
gen = vec(data["gen"][:, 2]) / 100

m = sum(mg) / trapz((yrange, xrange), isinside) .* isinside
d = (sum(dg) + sum(dl)) / trapz((yrange, xrange), isinside) .* isinside

trapz((yrange, xrange), m)
trapz((yrange, xrange), d)

pl = zeros(Ny, Nx)
pg = zeros(Ny, Nx)

b = 10
sigma = 100.

Threads.@threads for g in 1:length(idgen)
    Threads.@threads for i=2:Ny-1
        Threads.@threads for j=2:Nx-1
            idg1 = coord[idgen[g],1]
            idg2 = coord[idgen[g],2]
            n = norm_dist([idg1, idg2], [xrange[j], yrange[i]], sigma)
            pg[i,j] += gen[g] * n
        end
    end
end

Threads.@threads for l in 1:length(dl)
    Threads.@threads for i=2:Ny-1
        Threads.@threads for j=2:Nx-1
            n = norm_dist(vec(coord[l,:]), [xrange[j], yrange[i]], sigma)
            pl[i,j] += dem[l] * n
        end
    end
end
pl = sum(dem) .* pl ./ trapz((yrange, xrange), pl)
pg = sum(gen) .* pg ./ trapz((yrange, xrange), pg)
p = pg - pl 
p[.!isinside] .= 0 

p[isinside] = p[isinside] .- sum(p[isinside]) / sum(isinside)

In [ ]:
# compute, iteratively, the stable solution
interval = 1000
Niter = 40000

th0 = zeros(Ny, Nx)

@time begin
    for t in 1:Niter
        if(mod(t, interval) == 0)
            th_elder = copy(th0)
        end
        Threads.@threads for i in 2:Ny-1
            Threads.@threads for j in 2:Nx-1
                if(isinside[i,j])
                    bij = 4*b
                    th0[i,j] = (b * th0[i+1,j] + b * th0[i-1,j] + 
                        b * th0[i,j+1] + b * th0[i,j-1] + dx^2*p[i,j]) / bij
                end
            end
        end
        
        #Threads.@threads for k in 1:size(n,1)
        for k in 1:size(n,1)
            i = Int64(n[k,1])
            j = Int64(n[k,2])
            nx = n[k,4]
            ny = n[k,3]
            if(nx == 1)
                th0[i,j] = th0[i,j-2]
            else(nx == -1)
                th0[i,j] = th0[i,j+2]
            end
            if(ny == 1)
                th0[i,j] = th0[i-2,j]
            else(ny == -1)
                th0[i,j] = th0[i+2,j]
            end
        end
        if(mod(t, interval) == 0)
            println([t maximum(abs.(th0-th_elder))])
        end

    end
end

In [ ]:
contour(th0,fill=true)
#maximum(abs.(th0))

In [ ]:
# perform a dynamical simulation

interval = 500
dt = 0.001
Ndt = 120000

#interval = 1
#dt = 0.001
#Ndt = 4

gamma = d ./ m
#gamma .*= 10 

omegas = zeros(Ny,Nx,1 + Int64(ceil(Ndt/interval)))
thetas = zeros(Ny,Nx,1 + Int64(ceil(Ndt/interval)))
th_new = zeros(Ny,Nx)
th_old = copy(th0)
th = copy(th0)
omegas[:,:,1] = (th - th0) / dt
thetas[:,:,1] = copy(th)    
ts = zeros(1 + Int64(ceil(Ndt/interval)))
chi = 1 .+ gamma*dt/2

@time begin
    for k in 1:Ndt
        Threads.@threads for i in 2:Ny-1
            Threads.@threads for j in 2:Nx-1
                if(isinside[i,j])
                    bij = 4. * b
                    th_new[i,j] = (2. - bij / m[i,j] * dt^2 / dx^2) / chi[i,j] * th[i,j] + 
                        (gamma[i,j] * dt/2. - 1.) /chi[i,j] * th_old[i,j] + 
                        dt^2 / dx^2 / chi[i,j] / m[i,j] * 
                        (b * th[i+1,j] + b * th[i-1,j] +
                        b * th[i,j+1] + b * th[i,j-1]) +
                        dt^2 / chi[i,j] / m[i,j] * (p[i,j])
                end
            end
        end
        # impose boundary condition
        Threads.@threads for k in 1:size(n,1)
            i = Int64(n[k,1])
            j = Int64(n[k,2])
            nx = n[k,4]
            ny = n[k,3]
            if(nx == 1)
                th_new[i,j] = th_new[i,j-2]
            else(nx == -1)
                th_new[i,j] = th_new[i,j+2]
            end
            if(ny == 1)
                th_new[i,j] = th_new[i-2,j]
            else(ny == -1)
                th_new[i,j] = th_new[i+2,j]
            end
        end

        if(mod(k,interval) == 0)
            println("NIter: ", k)
            omegas[:,:,Int64(k/interval) + 1] = (th_new-th) / dt
            thetas[:,:,Int64(k/interval) + 1] = th_new
            ts[Int64(k/interval) + 1] = k*dt
        end
        th_old = copy(th)
        th = copy(th_new)
    end
end

In [ ]:
@gif for i=1:size(omegas,3)
    do_plot(isinside,omegas[:,:,i])
end

In [ ]:
plot(omegas[60,100,1:end])
plot!(omegas[10,100,1:end])
plot!(omegas[20,20,1:end])